In [1]:
import shutil
try:
    shutil.rmtree('Data')
    shutil.rmtree('output')
except:
    print('Directory doesnt exist')

# Downloading data

In [2]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='mLfby7c4EdgN_6a6QZJqLwTpK64eC2k3GcMHPdW-VC91',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'digitalnaturalistmodel-donotdelete-pr-u6twiyt8iaxoqc'
object_key = 'Digital Naturalist Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


# Unzipping data folder

In [3]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

# Data augmentation

In [4]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
from os import listdir
import time

In [5]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"

In [6]:
def augment_data(file_dir, n_generated_samples, save_to_dir):
    data_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.15, shear_range=0.25, zoom_range=0.2, horizontal_flip=True, vertical_flip=False, fill_mode='nearest', brightness_range=(0.5,1.2))
    for filename in listdir(file_dir):
        image = cv2.imread(file_dir + '/' + filename)
        image = image.reshape((1,) + image.shape)
        save_prefix = 'aug_' + filename[:-4]
        i = 0
        for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir,save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i>n_generated_samples:
                break

## Passing input to augment_data ()

In [7]:
try:
    os.mkdir('Data')
    os.mkdir('Data/Bird-Great Indian Bustard Bird')
    os.mkdir('Data/Bird-Spoon Billed Sandpiper Bird')
    os.mkdir('Data/Mammal-Pangolin Mammal')
    os.mkdir('Data/Mammal-Senenca White Deer Mammal')
    os.mkdir('Data/Flower-Corpse Flower')
    os.mkdir('Data/Flower-Lady Slipper Orchid Flower')
except:
    print('Directory already exists')

In [8]:
temp = """path1 = '/home/wsuser/work/Digital Naturalist Dataset/'
path2 = '/home/wsuser/work/Data/'
for dirc in os.listdir(path1):
    for dirk in os.listdir(path1+dirc):
        for pic in os.listdir(path1+dirc+'/'+dirk):
            shutil.copy(path1+dirc+'/'+dirk+'/'+pic,path2+dirc+'-'+dirk+'/'+pic)"""

In [9]:
start_time = time.time()
augmented_data_path = '/home/wsuser/work/Data/'

#Birds
augment_data(file_dir='/home/wsuser/work/Digital Naturalist Dataset/Bird/Great Indian Bustard Bird' , n_generated_samples=8, save_to_dir=augmented_data_path+'Bird-Great Indian Bustard Bird')
augment_data(file_dir='/home/wsuser/work/Digital Naturalist Dataset/Bird/Spoon Billed Sandpiper Bird' , n_generated_samples=8, save_to_dir=augmented_data_path+'Bird-Spoon Billed Sandpiper Bird')
 
#Mammals
augment_data(file_dir='/home/wsuser/work/Digital Naturalist Dataset/Mammal/Pangolin Mammal' , n_generated_samples=8, save_to_dir=augmented_data_path+'Mammal-Pangolin Mammal')
augment_data(file_dir='/home/wsuser/work/Digital Naturalist Dataset/Mammal/Senenca White Deer Mammal' , n_generated_samples=8, save_to_dir=augmented_data_path+'Mammal-Senenca White Deer Mammal')

#Flowers
augment_data(file_dir='/home/wsuser/work/Digital Naturalist Dataset/Flower/Corpse Flower' , n_generated_samples=8, save_to_dir=augmented_data_path+'Flower-Corpse Flower')
augment_data(file_dir='/home/wsuser/work/Digital Naturalist Dataset/Flower/Lady Slipper Orchid Flower' , n_generated_samples=8, save_to_dir=augmented_data_path+'Flower-Lady Slipper Orchid Flower')

end_time = time.time()
execution_time = end_time - start_time
print(f"Elapsed Time: {hms_string(execution_time)}")

Elapsed Time: 0:0:13.9


# Train-Test Split

In [10]:
%pip install split-folders
import splitfolders
splitfolders.ratio('/home/wsuser/work/Data/', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Note: you may need to restart the kernel to use updated packages.


Copying files: 1250 files [00:00, 7753.86 files/s]


# Image Augmentation

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
train_generator = ImageDataGenerator()
test_generator = ImageDataGenerator()

In [13]:
x_train = train_generator.flow_from_directory(r"/home/wsuser/work/output/train", target_size = (64,64), class_mode = 'categorical', batch_size = 24)
x_val = train_generator.flow_from_directory(r"/home/wsuser/work/output/val", target_size = (64,64), class_mode = 'categorical', batch_size = 24)
x_test = test_generator.flow_from_directory(r"/home/wsuser/work/output/test", target_size = (64,64), class_mode = 'categorical', batch_size = 24)

Found 998 images belonging to 6 classes.
Found 121 images belonging to 6 classes.
Found 131 images belonging to 6 classes.


In [14]:
x_train.class_indices

{'Bird-Great Indian Bustard Bird': 0,
 'Bird-Spoon Billed Sandpiper Bird': 1,
 'Flower-Corpse Flower': 2,
 'Flower-Lady Slipper Orchid Flower': 3,
 'Mammal-Pangolin Mammal': 4,
 'Mammal-Senenca White Deer Mammal': 5}

In [15]:
ls -n output/val/'Bird-Great Indian Bustard Bird'

total 192
-rw-rw---- 1 1000 103000 8429 Nov 11 18:17  aug_download_0_6556.jpg
-rw-rw---- 1 1000 103000 9502 Nov 11 18:17  aug_download_0_7120.jpg
-rw-rw---- 1 1000 103000 8300 Nov 11 18:17 'aug_download (10)_0_9528.jpg'
-rw-rw---- 1 1000 103000 8601 Nov 11 18:17 'aug_download (1)_0_5090.jpg'
-rw-rw---- 1 1000 103000 9041 Nov 11 18:17 'aug_download (2)_0_4751.jpg'
-rw-rw---- 1 1000 103000 7299 Nov 11 18:17 'aug_download (3)_0_9637.jpg'
-rw-rw---- 1 1000 103000 8840 Nov 11 18:17 'aug_download (6)_0_130.jpg'
-rw-rw---- 1 1000 103000 6994 Nov 11 18:17 'aug_download (7)_0_8398.jpg'
-rw-rw---- 1 1000 103000 7159 Nov 11 18:17 'aug_download (7)_0_847.jpg'
-rw-rw---- 1 1000 103000 9397 Nov 11 18:17 'aug_download (8)_0_9654.jpg'
-rw-rw---- 1 1000 103000 4593 Nov 11 18:17 'aug_download (9)_0_1290.jpg'
-rw-rw---- 1 1000 103000 7442 Nov 11 18:17 'aug_images (1)_0_1910.jpg'
-rw-rw---- 1 1000 103000 4959 Nov 11 18:17 'aug_images (2)_0_4637.jpg'
-rw-rw---- 1 1000 103000 9277 Nov 11 18:17 'aug_images (

# Creating the model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import regularizers

In [82]:
model = Sequential() #Initializing sequential model3

# Adding layers

In [83]:
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))  #Convolution layer
model.add(MaxPooling2D(pool_size=(2,2)))  #MaxPooling layer
model.add(Flatten())  #Flatten layer
model.add(Dense(275,activation='relu',kernel_regularizer=regularizers.L2(0.1)))  #Hidden layer 1
model.add(Dense(130,activation='relu',kernel_regularizer=regularizers.L2(0.01)))  #Hidden layer 2
model.add(Dense(50,activation='relu',kernel_regularizer=regularizers.L2(0.001))) #Hidden layer 3
model.add(Dense(6,activation='softmax')) #Output layer

# Compile the Model

In [84]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 30752)             0         
                                                                 
 dense_40 (Dense)            (None, 275)               8457075   
                                                                 
 dense_41 (Dense)            (None, 130)               35880     
                                                                 
 dense_42 (Dense)            (None, 50)                6550      
                                                     

In [85]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_accuracy',patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',patience=3,factor=0.5,min_lr=0.00001)

callback = [reduce_lr,early_stopping]

In [86]:
model.fit(x_train, steps_per_epoch=len(x_train), epochs=30, validation_data=x_val, validation_steps=len(x_val), callbacks = callback)

Epoch 1/30
42/42 [==============================] - 7s 153ms/step - loss: 213.0748 - accuracy: 0.3647 - val_loss: 48.7334 - val_accuracy: 0.4876 - lr: 0.0010
Epoch 2/30
42/42 [==============================] - 6s 152ms/step - loss: 39.7150 - accuracy: 0.6323 - val_loss: 35.5026 - val_accuracy: 0.5124 - lr: 0.0010
Epoch 3/30
42/42 [==============================] - 6s 151ms/step - loss: 29.4368 - accuracy: 0.7926 - val_loss: 27.4529 - val_accuracy: 0.5372 - lr: 0.0010
Epoch 4/30
42/42 [==============================] - 6s 152ms/step - loss: 23.3615 - accuracy: 0.8778 - val_loss: 22.7443 - val_accuracy: 0.5207 - lr: 0.0010
Epoch 5/30
42/42 [==============================] - 6s 152ms/step - loss: 19.1763 - accuracy: 0.9339 - val_loss: 19.2045 - val_accuracy: 0.6198 - lr: 0.0010
Epoch 6/30
42/42 [==============================] - 7s 156ms/step - loss: 16.2808 - accuracy: 0.9309 - val_loss: 16.4855 - val_accuracy: 0.6116 - lr: 0.0010
Epoch 7/30
42/42 [==============================] - 7s 15

In [87]:
a = model.evaluate_generator(x_test, len(x_test))

/tmp/wsuser/ipykernel_64086/1516118933.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  a = model.evaluate_generator(x_test, len(x_test))


In [88]:
print(f'Error: {a[0]}')
print(f'Accuracy: {a[1]}')

Error: 12.807180404663086
Accuracy: 0.6259542107582092


In [67]:
model.save('model1.h5')